In [1]:
%%bash
cd Attack3
make

g++ -no-pie -march=native -mtune=native -O3 -std=gnu++11 -Wall -I/usr/include/eigen3 -g   -c -o demo.o demo.cpp
g++ -no-pie -march=native -mtune=native -O3 -std=gnu++11 -Wall -I/usr/include/eigen3 -g   -c -o Entropy.o Entropy.cpp
g++ -no-pie -march=native -mtune=native -O3 -std=gnu++11 -Wall -I/usr/include/eigen3 -g   -c -o Fft.o Fft.cpp
g++ -no-pie -march=native -mtune=native -O3 -std=gnu++11 -Wall -I/usr/include/eigen3 -g   -c -o KeyGen.o KeyGen.cpp
g++ -no-pie -march=native -mtune=native -O3 -std=gnu++11 -Wall -I/usr/include/eigen3 -g   -c -o Sampler.o Sampler.cpp
g++ -no-pie -march=native -mtune=native -O3 -std=gnu++11 -Wall -I/usr/include/eigen3 -g   -c -o Setup.o Setup.cpp
g++ -no-pie -march=native -mtune=native -O3 -std=gnu++11 -Wall -I/usr/include/eigen3 -g   -c -o Sign.o Sign.cpp
g++ -no-pie -march=native -mtune=native -O3 -std=gnu++11 -Wall -I/usr/include/eigen3 -g   -c -o Verify.o Verify.cpp
g++ -no-pie -o demo demo.o Entropy.o Fft.o KeyGen.o Sampler.o Setup.o Sign.o Verify.

# Attack 3

## 1) Profiling phase

In [2]:
import sys
import os
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
import sklearn.preprocessing
import seaborn as sns
import matplotlib.pyplot as plt

from access_data import read_gaussian_data, read_bliss_data

In [3]:
# Path to the extracted data from Zenodo
DATA_PATH = "../galactics_attack_data/"

In [4]:
def series_to_numpy(s):
    assert len(s), 's must contain at least one row'
    l1, l2, t = len(s), len(next(iter(s))), next(iter(s)).dtype
    res = np.empty(shape=(l1, l2))
    for idx, row in enumerate(s):
        res[idx] = row
    return res

## 2) Attack Phase

In [5]:
data = read_bliss_data(bliss_dir=DATA_PATH, data_dir="data_device_b_attack_3/", attack3=True)

In [6]:
# filter b to relevant bit
# if b==1 then 
data['b'] = data.apply(lambda row: (row['b'] % 2), axis=1)

In [7]:
sc_cflip_data = pd.read_pickle(DATA_PATH + "data_device_b_attack_3/sc_cflip_cw_data.pickle")

Each trace consists of 232 samples. Only the samples [TRACE_SECTION_START, TRACE_SECTION_END] are of interest for the side-channel leakage.

In [8]:
#we will not use data['b'], instead we will use the predicted b by the classifier 
sc_cflip_data['trace'] = sc_cflip_data.apply(lambda row: row['trace'][30:80], axis=1)
sc_cflip_data['b'] = sc_cflip_data.apply(lambda row: int(row['inputs'][1]%2), axis=1) 

In [9]:
import pickle

with open('models/b.pickle', 'rb') as f:
    sc_cflip_clf = pickle.load(f)

In [10]:
sc_cflip_data['predictions'] = sc_cflip_clf.predict(sklearn.preprocessing.StandardScaler().fit_transform(series_to_numpy(sc_cflip_data['trace'])))

In [11]:
sc_predictions = -np.array(sc_cflip_data['predictions'])

In [12]:
sc_cflip_data['predictions'] = sc_cflip_clf.predict(sklearn.preprocessing.StandardScaler().fit_transform(series_to_numpy(sc_cflip_data['trace'])))
count = 0
for i in range(300000):
    if(sc_predictions[2*i]==data['b'][i] ):
        count=count+1
        
print(count)

211


In [13]:
def tw_file(s, fname):
    series_to_numpy(s).flatten().tofile(fname, sep=',', format='%i')

In [14]:
tw_file(data['z1'], 'Attack3/z1_file.txt')

In [15]:
tw_file(data['z2'], 'Attack3/z2_file.txt')

In [16]:
tw_file(data['c'], 'Attack3/c_file.txt')

In [17]:
s1 = data.iloc[0]['s1']
s1.tofile('Attack3/s1_file.txt', sep=',', format='%i')

In [18]:
s2 = data.iloc[0]['s2']
s2.tofile('Attack3/s2_file.txt', sep=',', format='%i')

In [19]:
sc_predictions[::2].tofile('Attack3/b_file.txt', sep=',', format='%i')

The secret key can be recovered running `./demo` in `Attack3/` directory.

**It is recommended to run it in the terminal as the output might break the notebook.**

In [20]:
%%bash
#cd Attack3 && ./demo

The `./demo` outputs the secret key using the side-channel data. Inserting the secret key into the cell below proves that the result is correct.

In [21]:
(np.array(list(map(lambda s: int(s), filter(lambda s: s, map(lambda s: s.strip(), """0  1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 -1  1  1  0 -1 -1  1  0  0  1  0  1 -1  0  0  1 -1 -1  0  0  0 -1 
        0  1  1  0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0  0  0  0  1  0  0  1  0  1  0  0  0  1  0  0  0  0  0 -1  0 -1 
        0  0 -1  0  0  0  0  0  0  1 -1  0  0  0 -1 -1  0  0  0  0  0 -1  0  0  0  1  1  0  0  0  0  1  0  0  1  1  0 -1 -1 
        0  0  0  0  0  0  0 -1 -1  0  0  0  1  0  0  0 -1  1 -1  0  0  0  0  1  0  1 -1  0  0  0  0  1  0  1  0  0  0 -1  0 
        0 -1  1  0 -1 -1  1  0 -1  0 -1  0  0  0  0  0  0  0  0 -1  1  0  0  0  0  1  1  0  0  0  1  0  0 -1  0  1  0  1  0 
        0  1  0  0  0  0  0  0  0  1  0  1 -1  0 -1  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0  0  0  0  0 -1 -1  0  0 
        0  0  0  0  1  0 -1  0  0  1  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  1  0  1  0  0 -1  0  0  1  0  0  0  0  1 
        0 -1  0  1  0  0  0  0  0  0  0  0  0  1  0  0 -1 -1  0  0 -1  0 -1 -1  0  0  1  0  0  0  0  0 -1 -1  0  1  0  0  0 
        0  0  0  0  1  0  1  1 -1  0  0  0  0  0  0 -1  0 -1 -1  0 -1  0  0  0  1  0  0  0  0  0  0  0  1  0  0  0  0  0 -1 
        0 -1  0  0  0  0  1  1  1  0  1  0  0 -1  0  0  1  0 -1  0  1  0 -1  0  1  0 -1 -1  0  0  0  1  0  0  0  0  0  1  0 
        0  0  0  0  0  0  0  0  0  0 -1  0  1  0  0  0  0  0  1  0 -1 -1  1  0  0  0  0  0  0  0  1  0  1  0  0  1  0  0 -1 
        1  0  0  0  0  0 -1  0  0  0  0  0  0  0  1  0  0 -1  0  0 -1  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  1 -1 -1 
        0  0 -1  1  0  0  0  0  1  0  0  1  0  0  1  1  0  0  0  0  0  1 -1  0  0  0  1 -1  0  0  0  0 -1  0  0  0 -1  0  0 
        0  0 -1  0  0 """.strip(' []').split(' ')))))) == s1).mean()

1.0